## Packages

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
from src.preprocessing import add_features

## Loading data

In [6]:
datatypes = {
    'product_id': 'uint32',
    'client_id': 'uint32',
    'sales_net': 'float64',
    'quantity': 'uint8',
    'order_channel': 'object',
    'branch_id': 'uint16'
}

In [7]:
df = pd.read_csv('data/transactions_dataset.csv', dtype=datatypes, sep=';')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63319315 entries, 0 to 63319314
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date_order     object 
 1   date_invoice   object 
 2   product_id     uint32 
 3   client_id      uint32 
 4   sales_net      float64
 5   quantity       uint8  
 6   order_channel  object 
 7   branch_id      uint16 
dtypes: float64(1), object(3), uint16(1), uint32(2), uint8(1)
memory usage: 2.5+ GB


In [9]:
df.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,order_channel,branch_id
0,2017-09-25,2017-09-25,2376333,188502,155.44320,3,online,2732
1,2017-09-25,2017-09-25,2520527,835089,16.39440,3,at the store,10279
2,2017-09-25,2017-09-25,2536756,1977896,365.76624,21,online,6184
3,2017-09-25,2017-09-25,3028673,598816,8.59878,201,at the store,4200
4,2017-09-25,2017-09-25,203377,2086861,1093.37400,3,by phone,7051


## Preprocessing

In [10]:
def dates_preprocessing(df): 
    df['date_order'] = pd.to_datetime(df['date_order'], format='%Y-%m-%d')
    df['date_invoice'] = pd.to_datetime(df['date_invoice'], format='%Y-%m-%d')
    return df

In [12]:
def order_channel_preprocessing(df):
    df.order_channel = df.order_channel.astype('category')
    return df


In [15]:
def add_features(df):
    # Add unit sale price
    df["unit_price"] = df["sales_net"] / df["quantity"]
    df.unit_price = df.unit_price.astype('int8')
    
    # Add flow : 1 means it is a purchase (increased stock), -1 it is a sale
    profitable_transactions = df['sales_net'] >= 0
    unprofitable_transactions = df['sales_net'] < 0
    df["stock_flow"] = 0
    df.loc[profitable_transactions, 'stock_flow'] = 1
    df.loc[unprofitable_transactions, 'stock_flow'] = -1
    df.stock_flow = df.stock_flow.astype('int8')
    
    df["quantity"] = df["quantity"] * df["stock_flow"]
    df.quantity = df.quantity.astype('int32')
    
    # Date features
    df["month_order"] = df["date_order"].dt.month
    df["order_invoice_delta"] = (df["date_invoice"] - df["date_order"]).dt.days
    return df